# 데이터 지수화 - 최종 개발 접안지수 일일 예측데이터 계산용
### 풍속 , 풍향 , 유의파고 + 파향, 가중치 재설정 , 강수시 출항 불가 조건 추가
### 예시데이터는 2021년 10월 18일 최신 예측데이터

In [4]:
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv('data/1018_data.csv', encoding='utf-8')

df = pd.DataFrame(data)

In [6]:
df.head()

,일시,풍속(m/s),풍향(deg),유의파고(m),파향(deg),강수량(mm)
0,1,2.2,0,0.31,90.0,0.0
1,2,1.0,45,0.28,90.0,0.0
2,3,1.9,135,0.27,90.0,0.0
3,4,2.5,180,0.25,90.0,0.0
4,5,2.9,180,0.23,90.0,0.0


In [7]:
df.isnull().sum()

일시         0
풍속(m/s)    0
풍향(deg)    0
유의파고(m)    0
파향(deg)    0
강수량(mm)    0
dtype: int64

In [98]:
# data2019_1 = df.fillna(method='bfill')
# data2019_1.head()

In [99]:
# data2019_1.isnull().sum()

In [8]:
data_today = df[['일시','풍속(m/s)','풍향(deg)','유의파고(m)','파향(deg)','강수량(mm)']]

In [9]:
data_today.describe()

,일시,풍속(m/s),풍향(deg),유의파고(m),파향(deg),강수량(mm)
count,79.000000,79.000000,79.000000,79.000000,79.000000,79.000000
mean,40.000000,5.278481,80.886076,0.629747,93.132911,0.365823
std,22.949219,3.478014,78.068672,0.460635,57.163828,0.968462
min,1.000000,0.800000,0.000000,0.190000,0.000000,0.000000
25%,20.500000,2.950000,45.000000,0.300000,45.000000,0.000000
50%,40.000000,4.200000,45.000000,0.480000,90.000000,0.000000
75%,59.500000,6.650000,135.000000,0.820000,90.000000,0.000000
max,79.000000,17.600000,315.000000,2.390000,247.500000,4.900000


In [10]:
#풍향
def func_winddeg_index(x) :
    if 160<x<=270:
        return 1
    elif 135<x<=160 or 270<x<=290:
        return 2
    elif 105<x<=135 or 290<x<=315:
        return 3
    elif 0<=x<=105 or 315<x<=360:
        return 4
    

#풍속 함수정의 -> 풍향에서 살짝 바꿈
def func_windspeed_index(x):
    if x<=4:
        return 4
    elif 4<x<=9:
        return 3
    elif 9<x<=14:
        return 2
    elif 14<x:
        return 1

#파고
def func_wave_index(x):
    if x<=0.5:
        return 4
    elif 0.5<x<=1.0:
        return 3
    elif 1.0<x<=2.0:
        return 2
    elif 2.0<x:
        return 1  

#파향
def func_wavedeg_index(x):
    if 0<=x<=110 or 290<x<=360:
        return 4
    elif 270<x<=290:
        return 3
    elif 110<x<=135 or 225<x<=270:
        return 2
    elif 135<x<=225:
        return 1
    
#강수량
def func_rain_index(x):
    if 0<x:
        return 0
    else: 
        return 1

In [48]:
print(func_rain_index(0))

5


In [11]:
# df = df[['일시','풍속(m/s)','풍향(deg)','유의파고(m)']]
data_today['wind_speed_index'] = data_today['풍속(m/s)'].apply(lambda x : func_windspeed_index(x))

In [12]:
data_today.head()

,일시,풍속(m/s),풍향(deg),유의파고(m),파향(deg),강수량(mm),wind_speed_index
0,1,2.2,0,0.31,90.0,0.0,4
1,2,1.0,45,0.28,90.0,0.0,4
2,3,1.9,135,0.27,90.0,0.0,4
3,4,2.5,180,0.25,90.0,0.0,4
4,5,2.9,180,0.23,90.0,0.0,4


In [13]:
data_today['wind_deg_index'] = data_today['풍향(deg)'].apply(lambda x : func_winddeg_index(x))

In [15]:
data_today['wave_index'] = data_today['유의파고(m)'].apply(lambda x : func_wave_index(x))

In [16]:
data_today['wave_deg_index'] = data_today['파향(deg)'].apply(lambda x : func_wavedeg_index(x))

In [17]:
data_today['rain_index'] = data_today['강수량(mm)'].apply(lambda x : func_rain_index(x))

In [14]:
data_today.head()

,일시,풍속(m/s),풍향(deg),유의파고(m),파향(deg),강수량(mm),wind_speed_index,wind_deg_index,wave_index,wave_deg_index,rain_index
0,1,2.2,0,0.31,90.0,0.0,4,4,4,4,1
1,2,1.0,45,0.28,90.0,0.0,4,4,4,4,1
2,3,1.9,135,0.27,90.0,0.0,4,3,4,4,1
3,4,2.5,180,0.25,90.0,0.0,4,1,4,4,1
4,5,2.9,180,0.23,90.0,0.0,4,1,4,4,1


In [29]:
data2019_2.describe()

,풍속(m/s),풍향(deg),유의파고(m),파향(deg),강수량(mm),wind_speed_index,wind_deg_index,wave_index,wave_deg_index,rain_index
count,8670.000000,8670.000000,8670.000000,8670.000000,8670.000000,8670.000000,8670.000000,8670.000000,8670.000000,8670.000000
mean,5.913437,179.705767,1.239619,138.121107,0.055998,3.138524,2.553057,2.546021,3.007266,0.964014
std,3.208576,101.309299,0.810221,104.574984,0.532924,0.708421,1.344685,0.935523,1.310162,0.186266
min,0.000000,0.000000,0.100000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,3.500000,87.000000,0.700000,45.000000,0.000000,3.000000,1.000000,2.000000,2.000000,1.000000
50%,5.500000,192.000000,1.000000,115.000000,0.000000,3.000000,3.000000,3.000000,4.000000,1.000000
75%,8.000000,254.000000,1.600000,212.000000,0.000000,4.000000,4.000000,3.000000,4.000000,1.000000
max,20.500000,360.000000,6.200000,360.000000,22.500000,4.000000,4.000000,4.000000,4.000000,1.000000


### 가중치 설정 
#### 풍속 : 4.5점 / 파고 : 4점 / 풍향 :3점 / 파향 : 4.5점 총합 16 -> 각 점수를 총합으로 나눔
#### 풍속 : 0.28 / 파고 : 0.25 / 풍향 : 0.19점 / 파향 : 0.28 100점 은1.06
#### 만약 강수량이 존재하면 접안지수는 0

In [15]:
data_today['berthing_index'] = (data_today['wind_speed_index']*0.28) + (data_today['wind_deg_index']*0.19) + (data_today['wave_index']*0.25) + (data_today['wave_deg_index']*0.28)

In [16]:
data_today['berthing_index2'] = data_today['berthing_index']/4

In [17]:
data_today.head()

,일시,풍속(m/s),풍향(deg),유의파고(m),파향(deg),강수량(mm),wind_speed_index,wind_deg_index,wave_index,wave_deg_index,rain_index,berthing_index,berthing_index2
0,1,2.2,0,0.31,90.0,0.0,4,4,4,4,1,4.00,1.0000
1,2,1.0,45,0.28,90.0,0.0,4,4,4,4,1,4.00,1.0000
2,3,1.9,135,0.27,90.0,0.0,4,3,4,4,1,3.81,0.9525
3,4,2.5,180,0.25,90.0,0.0,4,1,4,4,1,3.43,0.8575
4,5,2.9,180,0.23,90.0,0.0,4,1,4,4,1,3.43,0.8575


In [18]:
output_file="data/berthing_index_1018_today.csv"
data_today.to_csv(output_file, index=False)

In [68]:
# df_rs = df[df['name'].str.contains('li')]

contains_berthing_time2 = data2018_2[data2018_2['일시'].str.contains(' 9:00|10:00|12:00|13:00|14:00|15:00|17:00')]

In [69]:
output_file="data/berthing_index_2018_2.csv"
contains_berthing_time2.to_csv(output_file, index=False)